<a href="https://colab.research.google.com/github/Satyasri03/FMML-2024/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [2]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [3]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [4]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [5]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [6]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [7]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [8]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [9]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [10]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [11]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [12]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [13]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
A.Increasing the percentage of the validation set:

Generally leads to a decrease in validation accuracy for the Nearest Neighbour algorithm. This happens because a larger validation set means a smaller training set, and the model has less data to learn from.
For the Random Classifier, the accuracy remains largely unaffected as it randomly assigns labels.
Reducing the percentage of the validation set:

This usually leads to an increase in validation accuracy for the Nearest Neighbour algorithm because the model has more training data, leading to better generalization.
As before, the Random Classifier's accuracy remains mostly the same due to its random nature.
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
A.Larger training set: A larger training set generally leads to better model performance and, consequently, a more accurate prediction of test accuracy using the validation set. This is because the model has more data to learn from and can better generalize to unseen data.

Larger validation set: A larger validation set provides a more reliable estimate of model performance, leading to a more accurate prediction of test accuracy. However, it also means a smaller training set, which might negatively impact the model's performance.

The balance: Finding a good balance between the sizes of the training and validation sets is crucial. A common split is 80% for training and 20% for validation, but this can be adjusted based on the specific dataset and task.
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?
A.A good starting point is to use a 70-30 split (70% training and 30% validation). This split often provides a reasonable balance between having enough data for training and having a sufficiently large validation set for a reliable performance estimate.

However, it's important to consider the following factors:

Dataset size: For very large datasets, a smaller validation set might be sufficient.
Model complexity: More complex models might require larger training sets to avoid overfitting.
Ultimately, the best percentage for the validation set depends on your specific dataset and model. You can experiment with different splits and use techniques like cross-validation to find the optimal balance.

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

In [16]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


X_train = traindata
y_train = trainlabel
X_test = testdata
y_test = testlabel


knn_1 = KNeighborsClassifier(n_neighbors=1)
knn_1.fit(X_train, y_train)


y_pred_1 = knn_1.predict(X_test)
accuracy_1 = accuracy_score(y_test, y_pred_1)


knn_3 = KNeighborsClassifier(n_neighbors=3)
knn_3.fit(X_train, y_train)

y_pred_3 = knn_3.predict(X_test)
accuracy_3 = accuracy_score(y_test, y_pred_3)
print("Accuracy of 1-Nearest Neighbor Classifier: {:.2f}".format(accuracy_1))
print("Accuracy of 3-Nearest Neighbor Classifier: {:.2f}".format(accuracy_3))

Accuracy of 1-Nearest Neighbor Classifier: 0.34
Accuracy of 3-Nearest Neighbor Classifier: 0.35


## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [14]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [15]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
A.Yes, averaging validation accuracy across multiple splits generally gives more consistent results. This technique, known as cross-validation, helps to reduce the influence of any particular random split on the estimated performance. By averaging the results across multiple splits, you get a more robust and reliable estimate of how well your model generalizes to unseen data.
2. Does it give more accurate estimate of test accuracy?
A.Yes, averaging validation accuracy through techniques like cross-validation typically gives a more accurate estimate of test accuracy compared to using a single validation split. This is because:

Reduced bias: Cross-validation helps reduce the bias that can arise from a single, potentially unrepresentative, validation split.
Better representation: By using multiple splits, you ensure that different parts of the data are used for validation, giving a more comprehensive view of model performance.
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
A.Generally, increasing the number of iterations in cross-validation leads to a more accurate and stable estimate of model performance. Each iteration provides more data points to average, reducing the impact of outliers and random variations in the data splits. However, there are diminishing returns as the number of iterations increases. After a certain point, the improvement in accuracy becomes marginal, while the computational cost continues to rise.
4.Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?
A.While increasing iterations in cross-validation can improve the reliability of the estimate, it cannot fully compensate for a very small training or validation dataset. Here's why:

Small training set: With limited training data, the model may not learn the underlying patterns effectively, leading to poor generalization regardless of the number of iterations.
Small validation set: Even with many iterations, a small validation set might not adequately represent the overall data distribution, potentially leading to inaccurate performance estimates.
In such cases, consider techniques like data augmentation or transfer learning if applicable.

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.

In [17]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, neighbours=1):

    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = KNeighborsClassifier(n_neighbors=neighbours).fit(traindata, trainlabel).predict(valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations

In [18]:
splits = 10
splitpercent = 75/100

avg_acc_1 = AverageAccuracy(alltraindata, alltrainlabel, splitpercent, splits, neighbours=1)
print(f"Average validation accuracy for 1NN with {splits} splits and {splitpercent*100}% split: {avg_acc_1*100:.2f}%")

avg_acc_3 = AverageAccuracy(alltraindata, alltrainlabel, splitpercent, splits, neighbours=3)
print(f"Average validation accuracy for 3NN with {splits} splits and {splitpercent*100}% split: {avg_acc_3*100:.2f}%")

Average validation accuracy for 1NN with 10 splits and 75.0% split: 33.82%
Average validation accuracy for 3NN with 10 splits and 75.0% split: 34.75%


To observe how the accuracy of the 3 nearest neighbour classifier changes in relation to the number of splits and split size, and to compare it with the 1 nearest neighbour classifier, you'll need to conduct a series of experiments. Here's a suggested approach:


Experiment with different values:

    Number of splits: Try different values for the number of splits (e.g., 5, 10, 20).
        Split size: Vary the percentage of data used for training (e.g., 60/40, 70/30, 80/20).
        Neighbours: Compare the results for 1 nearest neighbour and 3 nearest neighbours.

    Run experiments and collect results:

For each combination of splits, split size, and neighbours, calculate the average accuracy using the AverageAccuracy function. Record the results in a table or plot them on a graph.

    Analyze the results:

    Effect of splits: How does increasing the number of splits affect the accuracy and consistency of the results for both classifiers?
        Effect of split size: How does changing the split size impact the accuracy of both classifiers?
        Comparison: How do the accuracies of the 1 nearest neighbour and 3 nearest neighbour classifiers compare across different splits and split sizes?